In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def createfiledirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

# **Define Functions to Load Bill XML Data Locally**

In [3]:
def creaturllist(searchpdf, tplist, ext, addcheck):
    typenapd = pd.DataFrame(columns=['Name', 'Link', 'OfficialTitle', 'BillStage', 'Date',  'LegisType', 'CommitteeName', 
                                    'SponsorLastName', 'SponsorFirstName', 'SponsorState',  'CoSponsorLastName', 'CoSponsorFirstName', 'CoSponsorState'])
    for page in searchpdf.pages:
        for annots in page.get("/Annots"):
            url=str(annots.get("/A").get("/URI"))
            if url not in typenapd['Link'].values and url.find(tplist)!=-1:
                typena=url.split(tplist)[1].split(ext)[0]
                if isinstance(addcheck, str):
                    typenapd=typenapd.append({'Name':typena, 'Link':url}, ignore_index=True)
                else:
                    if typena not in addcheck['Name'].values:
                        typenapd=typenapd.append({'Name':typena, 'Link':url}, ignore_index=True)
    return typenapd

In [4]:
def importfiles(filepd, filedir, typestr):   
    for ind in filepd.index:
        name=filepd.loc[ind, 'Name']
        link=filepd.loc[ind, 'Link']
        response = requests.get(link)
        file=filedir+ '/'+name+typestr
        with open(filedir + '/'+name+typestr, 'wb') as file:
            file.write(response.content)
            file.close()
    return 

# **Call Functions to Load Bill XML Data Locally** 

In [6]:
username='joythompson'
rawdf="/Users/"+username+"/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/Raw_Data_Files/"
searchpdf = pikepdf.Pdf.open(rawdf+ "search.pdf")   

directorymid=createfiledirectory(rawdf+ 'Bill_Data_from_HTML_XML')
directoryxml=createfiledirectory( directorymid+ '/DSE_project_xml')
directorytext=createfiledirectory(directorymid+'/DSE_project_text')

In [8]:
xmlnames=creaturllist(searchpdf, '/xml/', '.xml', 'None')
textnames=creaturllist(searchpdf, '/html/', '.htm', xmlnames)

importfiles(xmlnames, directoryxml, '.xml')
importfiles(textnames, directorytext, '.text')

xmlnames.to_csv(directorymid+ '/bill_attributes_via_xml_dataframe_only_index.csv')
textnames.to_csv(directorymid+'/bill_attributes_via_text_dataframe_only_index.csv')